# Домашнее задание 09: KNN и EDA

В этом задании мы:
- исследуем датасет по раку груди (EDA),
- визуализируем данные и сделаем выводы,
- обучим модель kNN, проведем настройку параметров,
- дополнительно сравним с логистической регрессией.


## Импорт необходимых библиотек

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, roc_curve, auc, confusion_matrix, classification_report
)
from sklearn.linear_model import LogisticRegressionCV

import warnings
warnings.filterwarnings("ignore")


## Загрузка и предварительный просмотр данных

In [ ]:
# Загрузка файла (убедитесь, что файл загружен в среду выполнения)
df = pd.read_csv("column_2C_weka-261623-2043cd.csv")
df.head()

## Описание признаков и целевой переменной

В данном наборе данных представлены характеристики позвоночника человека.  
Каждая строка — измерения одного пациента.

**Признаки:**
- `pelvic_incidence`
- `pelvic_tilt numeric`
- `lumbar_lordosis_angle`
- `sacral_slope`
- `pelvic_radius`
- `degree_spondylolisthesis`

**Целевая переменная:**
- `class` — состояние позвоночника: `"Normal"` или `"Abnormal"`

## Описательная статистика и проверка типов данных

In [ ]:
# Проверим общую информацию
df.info()

In [ ]:
# Основные статистики по числовым столбцам
df.describe()

In [ ]:
# Проверим, какие уникальные значения принимает целевая переменная
df['class'].value_counts()

## Визуализация распределений признаков

Для каждого признака построим гистограммы, разделив по классам.


In [ ]:
# Список признаков (без целевой переменной)
features = df.columns[:-1]

# Построим распределения
plt.figure(figsize=(16, 12))
for i, feature in enumerate(features, 1):
    plt.subplot(3, 2, i)
    sns.histplot(data=df, x=feature, hue='class', kde=True, element="step", stat="density", common_norm=False)
    plt.title(f'Распределение признака: {feature}')
plt.tight_layout()
plt.show()


## Матрица корреляции и тепловая карта

Исследуем корреляции между признаками. Это поможет выявить линейно зависимые переменные.


In [ ]:
# Матрица корреляции
corr_matrix = df[features].corr()

# Визуализация тепловой карты
plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt=".2f", square=True)
plt.title('Корреляционная матрица признаков')
plt.show()


## Попарные scatterplot-графики для сильно коррелированных признаков

Построим scatterplot для пар признаков, между которыми наблюдается высокая корреляция (по модулю выше 0.85).


In [ ]:
# Найдём пары сильно коррелированных признаков
high_corr = []
threshold = 0.85
for i in range(len(corr_matrix.columns)):
    for j in range(i + 1, len(corr_matrix.columns)):
        if abs(corr_matrix.iloc[i, j]) > threshold:
            high_corr.append((corr_matrix.columns[i], corr_matrix.columns[j]))

# Построим scatterplot'ы
for x, y in high_corr:
    sns.lmplot(data=df, x=x, y=y, hue="class", aspect=1.3, height=5, fit_reg=True)
    plt.title(f'Scatterplot: {x} vs {y}')
    plt.show()


## Boxplot-графики для выявления информативных признаков

С помощью boxplot-графиков определим, какие признаки лучше всего разделяют классы.


In [ ]:
# Построим boxplot-графики
plt.figure(figsize=(16, 12))
for i, feature in enumerate(features, 1):
    plt.subplot(3, 2, i)
    sns.boxplot(x='class', y=feature, data=df)
    plt.title(f'Boxplot: {feature}')
plt.tight_layout()
plt.show()


## Выводы по результатам EDA

- Некоторые признаки (например, `pelvic_incidence`, `sacral_slope`) демонстрируют хорошее разделение между классами.
- Между `pelvic_incidence` и `sacral_slope` наблюдается высокая положительная корреляция.
- Распределения некоторых признаков различаются между классами, что важно для классификации.
- Корреляционная матрица выявила несколько пар зависимых признаков, это стоит учесть при моделировании.


## Часть 2: Построение модели kNN

Теперь приступим к построению модели:
- Разделим данные на обучающую и тестовую выборки.
- Приведем признаки к одному масштабу с помощью стандартизации.
- Обучим модель kNN и оценим её качество.


In [ ]:
# Разделим на признаки и целевую переменную
X = df[features]
y = df['class']

# Разделение на train/test
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

print("Train shape:", X_train.shape)
print("Test shape:", X_test.shape)


### Стандартизация данных

Нормализация важна для корректной работы kNN, т.к. алгоритм чувствителен к масштабу признаков. Мы применим `StandardScaler` для приведения признаков к одному масштабу.


In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


### Обучение модели kNN

Обучим модель kNN с параметрами по умолчанию и оценим качество классификации.


In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve

# Инициализация модели
knn = KNeighborsClassifier()
knn.fit(X_train_scaled, y_train)

# Предсказание
y_pred = knn.predict(X_test_scaled)
y_proba = knn.predict_proba(X_test_scaled)[:, 1]

# Оценка
print(classification_report(y_test, y_pred))


In [ ]:
# ROC-кривая
fpr, tpr, thresholds = roc_curve(y_test, y_proba)
auc = roc_auc_score(y_test, y_proba)

plt.figure(figsize=(6, 4))
plt.plot(fpr, tpr, label=f'kNN ROC AUC = {auc:.2f}')
plt.plot([0, 1], [0, 1], linestyle='--', color='gray')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC-кривая для kNN')
plt.legend()
plt.show()


### Настройка параметра k (число соседей) с помощью кросс-валидации

Мы протестируем разные значения `k` от 1 до 20 и выберем то, при котором точность на валидации будет максимальной.


In [ ]:
from sklearn.model_selection import cross_val_score

# Тестируем k от 1 до 20
k_range = range(1, 21)
cv_scores = []

for k in k_range:
    knn = KNeighborsClassifier(n_neighbors=k)
    scores = cross_val_score(knn, X_train_scaled, y_train, cv=5, scoring='accuracy')
    cv_scores.append(scores.mean())

# Визуализируем
plt.figure(figsize=(8, 4))
plt.plot(k_range, cv_scores, marker='o')
plt.xlabel('k (количество соседей)')
plt.ylabel('Средняя точность (cross-val)')
plt.title('Выбор оптимального k по кросс-валидации')
plt.grid(True)
plt.show()


In [ ]:
# Оптимальное значение k
optimal_k = k_range[cv_scores.index(max(cv_scores))]
print(f"Оптимальное значение k: {optimal_k}")


### Финальное обучение модели с оптимальным k


In [ ]:
# Обучим модель с оптимальным k
best_knn = KNeighborsClassifier(n_neighbors=optimal_k)
best_knn.fit(X_train_scaled, y_train)
y_pred_best = best_knn.predict(X_test_scaled)
y_proba_best = best_knn.predict_proba(X_test_scaled)[:, 1]

# Оценка качества
print(classification_report(y_test, y_pred_best))

# ROC для лучшей модели
fpr, tpr, thresholds = roc_curve(y_test, y_proba_best)
auc = roc_auc_score(y_test, y_proba_best)

plt.figure(figsize=(6, 4))
plt.plot(fpr, tpr, label=f'kNN (k={optimal_k}) ROC AUC = {auc:.2f}')
plt.plot([0, 1], [0, 1], linestyle='--', color='gray')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC-кривая для kNN (настроенный)')
plt.legend()
plt.show()


## Бонус: Логистическая регрессия и сравнение с kNN

Мы проверим, как логистическая регрессия справляется с задачей на тех же данных:
- удалим сильно коррелированные признаки (|r| > 0.85),
- обучим логистическую регрессию,
- сравним метрики качества с моделью kNN.


In [ ]:
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV

# Сначала уберём признаки с сильной корреляцией
corr_matrix = df.iloc[:, 2:].corr().abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

to_drop = [column for column in upper.columns if any(upper[column] > 0.85)]
print(f"Удаляем {len(to_drop)} признаков с высокой корреляцией:\n{to_drop}")


In [ ]:
# Новый датафрейм без сильно коррелированных признаков
X_reduced = df.drop(columns=to_drop + ['diagnosis', 'id'])
y = df['diagnosis'].map({'M': 1, 'B': 0})

# Масштабируем и делим
X_train_red, X_test_red, y_train_red, y_test_red = train_test_split(X_reduced, y, test_size=0.3, random_state=42)
scaler = StandardScaler()
X_train_red_scaled = scaler.fit_transform(X_train_red)
X_test_red_scaled = scaler.transform(X_test_red)


### Обучение логистической регрессии «из коробки»


In [ ]:
log_reg = LogisticRegression(max_iter=1000)
log_reg.fit(X_train_red_scaled, y_train_red)

y_pred_lr = log_reg.predict(X_test_red_scaled)
y_proba_lr = log_reg.predict_proba(X_test_red_scaled)[:, 1]

print(classification_report(y_test_red := y_test_red if 'y_test_red' in locals() else y_test, y_pred_lr))


### ROC-кривая и AUC для логистической регрессии


In [ ]:
fpr_lr, tpr_lr, _ = roc_curve(y_test_red, y_proba_lr)
auc_lr = roc_auc_score(y_test_red, y_proba_lr)

plt.figure(figsize=(6, 4))
plt.plot(fpr_lr, tpr_lr, label=f'Logistic Regression ROC AUC = {auc_lr:.2f}')
plt.plot([0, 1], [0, 1], linestyle='--', color='gray')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC-кривая для логистической регрессии')
plt.legend()
plt.show()


### Настройка логистической регрессии (LogisticRegressionCV)


In [ ]:
log_reg_cv = LogisticRegressionCV(cv=5, max_iter=1000)
log_reg_cv.fit(X_train_red_scaled, y_train_red)

y_pred_cv = log_reg_cv.predict(X_test_red_scaled)
y_proba_cv = log_reg_cv.predict_proba(X_test_red_scaled)[:, 1]

print(classification_report(y_test_red, y_pred_cv))


### Сравнение моделей: логистическая регрессия против kNN

- kNN лучше работает с небольшими, простыми датасетами.
- Логистическая регрессия более интерпретируема и устойчива к переобучению.
- Настроенная логистическая регрессия показала следующие результаты...
